In [ ]:
-------------vlog about the outliers--------------------

# ---------------------------------outlier visualiztion of all the columns----------------------

In [ ]:
# To check outliers 
import matplotlib.pyplot as plt


a4_dims = (11.7, 8.27)
fig, ax = plt.subplots(figsize=a4_dims)
g = sns.boxplot(data=combine_data,linewidth=2.5,ax=ax)
g.set_yscale("log")

# ------------------------outliers deletion------------------------

In [4]:
from sklearn.datasets import load_iris 
import pandas as pd 
bunch_Obj_iris = load_iris() 

df = pd.DataFrame(bunch_Obj_iris.data,columns=bunch_Obj_iris.feature_names) 
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [8]:
# def drop_outliers_IQR(df): 

#    q1=df.quantile(0.25)

#    q3=df.quantile(0.75)

#    IQR=q3-q1

#    not_outliers = df[~((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]

#    outliers_dropped = not_outliers.dropna().reset_index()

#    return outliers_dropped

# outlier_removed_df = drop_outliers_IQR(df) 
# outlier_removed_df 

def drop_outliers_IQR(df):
    q1 = df.quantile(0.25) 
    q3 = df.quantile(0.75)  
    
    iqr = q3 - q1 
    
    lower_bound = q1 - 1.5*iqr 
    upper_bound = q3 + 1.5*iqr  
    
    not_outliers_df = df[~((df < lower_bound) | (df > upper_bound))] 
    outliers_dropped = not_outliers_df.dropna().reset_index() 
    return outliers_dropped 
                    
outlier_removed_df = drop_outliers_IQR(df) 
outlier_removed_df 


,index,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0,5.1,3.5,1.4,0.2
1,1,4.9,3.0,1.4,0.2
2,2,4.7,3.2,1.3,0.2
3,3,4.6,3.1,1.5,0.2
4,4,5.0,3.6,1.4,0.2
...,...,...,...,...,...
141,145,6.7,3.0,5.2,2.3
142,146,6.3,2.5,5.0,1.9
143,147,6.5,3.0,5.2,2.0
144,148,6.2,3.4,5.4,2.3


# -----------------------dummy code for outlier imputation-----------------------------

In [311]:
import pandas as pd
l = [70,80,1000,100,2000,130,60,300,800,200,150,250]
l2 = [70,80,1000,100,2000,1300,6,300,800,200,150,250]
df = pd.DataFrame(l,columns=['col_1']) 
df['col_2'] = l2

In [312]:
df

,col_1,col_2
0,70,70
1,80,80
2,1000,1000
3,100,100
4,2000,2000
5,130,1300
6,60,6
7,300,300
8,800,800
9,200,200


# ------Custome code develop by me to fill the outliers by median-------

In [9]:
import pandas as pd
l = [70,80,1000,100,2000,130,60,300,800,200,150,250]
l2 = [70,80,1000,100,2000,1300,6,300,800,200,150,250]
df = pd.DataFrame(l,columns=['col_1']) 
df['col_2'] = l2

def taking_series(value,median,lower_bound,upper_bound):
        if value < lower_bound or value > upper_bound:
            return str(median) + '<---fitted by the median value---'
        else:
            return value
            
def taking_column(series): 
    median = series.median()
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75) 
    
    iqr = q3 - q1 

    lower_bound = q1 - 1.5*iqr 
    upper_bound = q3 + 1.5*iqr  
    
    return series.apply(taking_series,args=(median,lower_bound,upper_bound)) # args=(2,3,4), extra_kw=1
df.apply(taking_column)

,col_1,col_2
0,70,70
1,80,80
2,175.0<---fitted by the median value---,1000
3,100,100
4,175.0<---fitted by the median value---,225.0<---fitted by the median value---
5,130,1300
6,60,6
7,300,300
8,800,800
9,200,200


# ---------fitting the outliers by median with the help of masking technique--------

In [2]:
import pandas as pd
l = [70,80,1000,100,2000,130,60,300,800,200,150,250]
l2 = [70,80,1000,100,2000,1300,6,300,800,200,150,250]
df = pd.DataFrame(l,columns=['col_1']) 
df['col_2'] = l2

for column in df.columns:
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    
    lower_limit = Q1 - 1.5 * (Q3 - Q1)
    upper_limit = Q3 + 1.5 * (Q3 - Q1)
    
    outliers_mask = (df[column] < lower_limit) | (df[column] > upper_limit)# outlier mask
    
    df.loc[outliers_mask, column] = df[column].median() # <-------------- understand this
    
df

,col_1,col_2
0,70,70
1,80,80
2,175,1000
3,100,100
4,175,225
5,130,1300
6,60,6
7,300,300
8,800,800
9,200,200


# ----- I think this method is not working properly-----

In [11]:
import pandas as pd
l = [70,80,1000,100,2000,130,60,300,800,200,150,250]
l2 = [70,80,1000,100,2000,1300,6,300,800,200,150,250]
df = pd.DataFrame(l,columns=['col_1']) 
df['col_2'] = l2

for column in df.columns:

    # Assuming you have a DataFrame called 'df' with a column named 'Value' containing your data

    # Calculate the median of the non-outlier values
    median = df[column].median()

    # Identify the outliers using a condition, such as values greater than 3 standard deviations from the median
    outliers = df[column] > (median + 3 * df[column].std())
    print(f'outliers : {outliers}')
    # Replace the outlier values with the calculated median
    df.loc[outliers, column] = median
df

outliers : 0     False
1     False
2     False
3     False
4      True
5     False
6     False
7     False
8     False
9     False
10    False
11    False
Name: col_1, dtype: bool
outliers : 0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
Name: col_2, dtype: bool


,col_1,col_2
0,70,70
1,80,80
2,1000,1000
3,100,100
4,175,2000
5,130,1300
6,60,6
7,300,300
8,800,800
9,200,200


In [14]:
df=pd.read_csv("insurance.csv")
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [21]:
%%time 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

# from feature_engine import encoding


# df=pd.read_csv("ds_salaries.csv")

# create custom transformer
class DataCleaning(BaseEstimator, TransformerMixin):
    def __init__(self, df):
        self.df = df

    def fit(self, X, y = None):
        return self
    
    # This method is used for cleaning the data
    def transform(self, X):
        percentage_null_value = round(X.isnull().sum()/len(X)*100.00,2)
        big_missing_values_columns = percentage_null_value[percentage_null_value>=30]
        X = X.drop(columns=big_missing_values_columns.index)
        df_num=X.select_dtypes(include=np.number)
        df_cat=X.select_dtypes(exclude=np.number)
        df_cat=df_cat.fillna(df_cat.mode().iloc[0])
        df_num=df_num.fillna(df_num.mean())
        X = pd.concat([df_cat, df_num],axis=1)
        return X
      
class FeatureEngineering(BaseEstimator, TransformerMixin):
    
     # Outlier imputation code 
    def taking_series(self,value,median,lower_bound,upper_bound,series):
            if value < lower_bound or value > upper_bound:
                return median
            else:
                return value

    def taking_column(self,series): 
        median = series.median()
        q1 = series.quantile(0.25)
        q3 = series.quantile(0.75) 

        iqr = q3 - q1 

        lower_bound = q1 - 1.5*iqr 
        upper_bound = q3 + 1.5*iqr  

        return series.apply(self.taking_series,args=(median,lower_bound,upper_bound,series))

    def __init__(self,df,output_col,unwanted_col_name=None):
        self.df = df
        self.output_col = output_col
        self.unwanted_col_name = unwanted_col_name
        # calling the outlier function
        self.df = self.df.select_dtypes(include=np.number) 
        self.df.apply(self.taking_column)
        print('outlier imputation is done...')
        
    def fit(self, X, y = None):
        return self
    
    # This method is used for feature-engineering
    def transform(self, X, y = None):
        df_num=X.select_dtypes(include=np.number)  
        df_cat=X.select_dtypes(exclude=np.number)

        if not df_cat.empty:
            dummies=pd.get_dummies(df_cat,drop_first=True)
            X=pd.concat([X,dummies],axis=1)
            X=X.drop(columns=df_cat[df_cat.columns],axis=1) 
        X=X.dropna(axis=0)
        if self.unwanted_col_name:
            X=df.drop(columns=self.unwanted_col_name,axis=1)
    #     X = df.drop(output_col,axis=1)
    #     y = df[output_col]
    #     X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=10)

        return X
      

X=df.drop(columns=['charges'],axis=1)
y=df['charges']

# print(f'X.shape : {X.shape}')
# print(f'y.shape : {y.shape}')

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, train_size=0.7, random_state=10)

print('Please wait...')
# {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
lr = LinearRegression()
dtr = DecisionTreeRegressor()
rfr = RandomForestRegressor()
xgbr = XGBRegressor()
# print(f'parameters : {rfr.get_params().keys()}')
models ={
        "Linear Regression":[lr,[{'classifier__fit_intercept':[True,False], 
          'classifier__normalize':[True,False], 
          'classifier__copy_X':[True, False]}]],
         
#          'DecisionTree Regressor':[dtr,[{"classifier__criterion": ["mse", "mae"],
#                 "classifier__max_depth" : [5,9,12,15],
#                "classifier__min_samples_leaf":[5,8,10],
#                "classifier__max_features":[5,9,12,15]}]],
    
#         'RandomForest Regressor':[rfr,[{"classifier__criterion": ["mse", "mae"],
#                 'classifier__n_estimators' : [100,200,250,300,400,500],
#                 "classifier__max_depth" : [5,9,12,15],
#                "classifier__min_samples_leaf":[5,8,10],
#                "classifier__max_features":[5,9,12,15]}]],
    
#         'XGBReg Regressor':[xgbr,[{'classifier__learning_rate': [0.01,0.001,0.1], #so called `eta` value
#               'classifier__max_depth': [3,4,5,8,10],
#               'classifier__colsample_bytree': [0.7],
#               'classifier__n_estimators': [10,50,80,100]}]],
    
         
        }
data_cleaning_obj = DataCleaning(df)
feature_eng_obj = FeatureEngineering(df,'charges')

training_score,testing_score = {},{}

# build final pipeline
for name, model in models.items():
  
    pipeline = Pipeline([
        ('cleaning',data_cleaning_obj),
        ("feature_eng", feature_eng_obj),
        ('classifier', model[0]) 
    ])
    print(f'-----------------------{name}-----------------------')
    gridsearchcv = GridSearchCV(pipeline, model[1], cv=3) # Fit grid search
    best_model_lr = gridsearchcv.fit(X_train,y_train)
    
    y_pred_train = gridsearchcv.predict(X_train)
    y_pred_test = gridsearchcv.predict(X_test)
    
#     training_score = r2_score(y_train,y_pred_train)
#     testing_score = r2_score(y_test,y_pred_test)
    
    training_score[name] = r2_score(y_train,y_pred_train)
    
    testing_score[name] = r2_score(y_test,y_pred_test)
    
    print(f'r2_score at training data : {training_score[name]} ')
    print(f'r2_score at testing data : {testing_score[name]} ')
    
training_list = ['model_name',0]
testing_list = ['model_name',0]

for key,val in training_score.items():
    if val > training_list[1]:
        training_list[0],training_list[1] = key,val
print(f'{training_list[0]} : {training_list[1]} <------------Best Model for training dataset----------')

for key,val in testing_score.items():
    if val > testing_list[1]:
        testing_list[0],testing_list[1] = key,val
print(f'{testing_list[0]} : {testing_list[1]} <------------Best Model for testing dataset----------')

Please wait...
outlier imputation is done...
-----------------------Linear Regression-----------------------
outlier imputation is done...


RuntimeError: Cannot clone object FeatureEngineering(df=      age     bmi  children      charges
0      19  27.900         0  16884.92400
1      18  33.770         1   1725.55230
2      28  33.000         3   4449.46200
3      33  22.705         0  21984.47061
4      32  28.880         0   3866.85520
...   ...     ...       ...          ...
1333   50  30.970         3  10600.54830
1334   18  31.920         0   2205.98080
1335   18  36.850         0   1629.83350
1336   21  25.800         0   2007.94500
1337   61  29.070         0  29141.36030

[1338 rows x 4 columns],
                   output_col='charges'), as the constructor either does not set or modifies parameter df